In [2]:
import numpy as np
import math
import itertools
from typing import Iterable, Iterator, Tuple, TypeVar, Dict
from dataclasses import dataclass
import matplotlib.pyplot as plt
from scipy.stats import norm
import timeit
from scipy.optimize import brentq
from scipy.optimize import newton
from IPython.display import clear_output

from numba import jit

In [13]:
@jit(parallel=True)
def get_value(F0:float,T:float,alpha0:float,beta:float,nu:float,
    rho:float,  K: float, Nstep:int, Ntrace: int)->Tuple[float,float]:

    payoff = np.zeros(Ntrace)
    for itrace in range(Ntrace):
        #if itrace%10==0:
        #    clear_output(wait=True)
        #    print("Trace #" + str(itrace))

        F1 = F0
        F2 = F0
        alpha1 = alpha0
        alpha2 = alpha0
        t = 0
        dt = T/Nstep
        #dt_list = np.flip(np.diff(np.geomspace(1e-5, self.T, num=self.Nstep),prepend=[0]))
        X1 = np.random.randn(Nstep,1)
        X2 = np.random.randn(Nstep,1)
        Z=rho*X1+np.sqrt(1-rho**2)*X2
        F1list = np.zeros(Nstep)
        F2list = np.zeros(Nstep)
        for istep in range(Nstep):
            if F1<0 or F2 < 0:
                break
            F1 += alpha1*(F1**beta)*np.sqrt(dt)*X1[istep][0]
            alpha1 += nu*alpha1*np.sqrt(dt)*Z[istep][0]
            F1list[istep] = F1
            F2 += alpha2*(F2**beta)*np.sqrt(dt)*X1[istep][0]*(-1)
            alpha2 += nu*alpha2*np.sqrt(dt)*Z[istep][0]*(-1)
            F2list[istep] = F2
        #plt.plot(F1list,'b')


        payoff1 = np.maximum(F1 - K,0)
        payoff2 = np.maximum(F2 - K,0)

        payoff[itrace] = (payoff1 + payoff2)/2




    value = np.mean(payoff[~np.isnan(payoff)])
    value_std = np.std(payoff[~np.isnan(payoff)])



    return value, value_std/np.sqrt(Ntrace)

def bs_call(F0:float, T:float, vol:float)->float:
      d1 = (np.log(F0/K) + (0.5*vol**2)*T) / (vol*np.sqrt(T))
      d2 = d1 - vol * np.sqrt(T)
      return F0 * norm.cdf(d1) -  K*norm.cdf(d2)

def im_vol(target_price:float,F0:float, T:float)->float:
    MAX_ITERATIONS = 10000
    PRECISION = 1.0e-7
    sig_u, sig_d = 2, 0.2
    sig = 0.5
    price = bs_call(F0=F0, T=T, vol=sig)
    err = target_price - price
    count = 0
    while abs(err)> PRECISION and count < MAX_ITERATIONS:
        if err > 0:
            sig_d = sig
            sig = (sig_u + sig_d)/2
        else:
            sig_u = sig
            sig = (sig_d + sig)/2
        price = bs_call(F0=F0, T=T, vol=sig)
        err = target_price - price
        count += 1
    return sig #

In [17]:
F0 = 1.0
T = 2.0
alpha0 = 0.35
beta = 0.25
nu = 1.0
rho = 0.25
K = 1.8
Nstep = 5000
Ntrace = 5000

mean_value,std_value = get_value(F0 = F0, T = T, alpha0 = alpha0, beta = beta, nu = nu,
    rho = rho, K=K, Nstep=Nstep, Ntrace=Ntrace)
print("mean value is " + str(mean_value))
print("std is " + str(std_value))

vol = im_vol(target_price = mean_value,F0=F0, T=T)
print("implied vol is " + str(vol))

mean value is 0.06571940794110855
std is 0.004607533069895403
implied vol is 0.41968221664428706


In [15]:
vol_list = []
value_list = []
Klist = np.linspace(0.6,1.9,10)
for K in Klist:
    print("K =" +str(K))
    mean_value,std_value = get_value(F0 = F0, T = T, alpha0 = alpha0, beta = beata, nu = nu,
        rho = rho, K=K, Nstep=Nstep, Ntrace=Ntrace)
    value_list.append(mean_value)
    vol = im_vol(target_price = mean_value,F0=F0, T=T)
    vol_list.append(vol)

plt.plot(Klist,value_list)
plt.figure()
plt.plot(Klist,vol_list)



K =0.6


NameError: name 'beata' is not defined